In [2]:
#importing libraries

import cv2
import os
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser
import datefinder
from IPython.display import clear_output

In [3]:
imagesFolderPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset/Images'

In [4]:
def preProcessDataFrame(df, image):
    
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    #new columns for page width and height for further normalization
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    #adding columns for zonal info (4 Zone)
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [5]:
def verifyDate(text):
    
    dates = list(datefinder.find_dates(text, strict=True))
    
    if len(dates)==0:
        return False
    else:
        for date in dates:
            year = date.year
            presentYear = date.today().year
            if abs(year-presentYear)<=2:
                print(dates)
                return True
            else:
                return False

In [6]:
def findDate(dateDF):
    dates = []
    
    for index in dateDF.index:
        text = dateDF["text"][index]
        if verifyDate(text):
            dates.append(text)
    return dates

In [7]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = text.lower()
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [8]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [31]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if x2<x1 and y2<y1:
                if df["text"][index2].lower() in neighbours:
                    neighbours[df["text"][index2].lower()]=min(dist, neighbours[df["text"][index2].lower()])
                else:
                    neighbours[df["text"][index2].lower()]=dist
            if abs(block2-block)<=1:
                if df["text"][index2].lower() in neighbours:
                    neighbours[df["text"][index2].lower()]=min(dist, neighbours[df["text"][index2].lower()])
                else:
                    neighbours[df["text"][index2].lower()]=dist
        for n in neighbourFields:
            if n in neighbours:
                df[n][index]=neighbours[n]

In [32]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [33]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [34]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [35]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [36]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [37]:
os.chdir(imagesFolderPath)

In [38]:
imagePath = "1.jpeg"

In [39]:
df = create(imagePath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[datetime.datetime(2023, 3, 1, 0, 0)]
[datetime.datetime(2023, 2, 27, 0, 0)]
[datetime.datetime(2023, 3, 1, 0, 0)]
[datetime.datetime(2023, 2, 27, 0, 0)]


/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]


In [40]:
df

,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,dated,invoice,delivery,order,due,payment,tax,bill,receipt,issue
210,37,2,2100,779,172,37,86.337227,1-Mar-23,-519,2272,...,0,392.081624,415.018072,0,0,432.226792,0.0,0,0,0
215,37,2,2098,901,192,35,88.978600,27-Feb-23,-537,2290,...,0,429.516007,395.216396,0,0,393.405643,0.0,0,0,0
427,37,0,2100,779,172,37,86.337227,1-Mar-23,-519,2272,...,0,392.081624,415.018072,0,0,432.226792,0.0,0,0,0
433,37,0,2098,901,192,35,88.978600,27-Feb-23,-537,2290,...,0,429.516007,395.216396,0,0,393.405643,0.0,0,0,0


In [41]:
dfList = batchCreate(imagesFolderPath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

[datetime.datetime(2023, 12, 1, 0, 0)]
[datetime.datetime(2023, 12, 1, 0, 0)]


/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[datetime.datetime(2023, 3, 28, 0, 0)]
[datetime.datetime(2023, 3, 28, 16, 37)]
[datetime.datetime(2023, 3, 28, 0, 0)]
[datetime.datetime(2023, 1, 29, 0, 0), datetime.datetime(2027, 1, 28, 0, 0)]
[datetime.datetime(2023, 3, 28, 0, 0)]
[datetime.datetime(2023, 3, 28, 0, 0)]


/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


...

[datetime.datetime(2022, 10, 19, 0, 0)]
[datetime.datetime(2022, 11, 18, 0, 0)]
[datetime.datetime(2022, 1, 9, 0, 0)]
[datetime.datetime(2022, 9, 30, 0, 0)]
[datetime.datetime(2022, 10, 19, 0, 0)]
[datetime.datetime(2022, 10, 19, 0, 0)]
[datetime.datetime(2022, 11, 18, 0, 0)]
[datetime.datetime(2022, 1, 9, 0, 0)]
[datetime.datetime(2022, 9, 30, 0, 0)]
[datetime.datetime(2022, 10, 19, 16, 51)]


/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

[datetime.datetime(2023, 1, 14, 0, 0)]
[datetime.datetime(2023, 1, 14, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5822/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5822/203255860.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

[datetime.datetime(2022, 4, 11, 0, 0)]
[datetime.datetime(2022, 4, 11, 0, 0)]


In [42]:
len(dfList)

90

In [48]:
resultDF = pd.concat(dfList)

In [49]:
resultDF.shape

(193, 30)

In [50]:
len(resultDF["imageName"].unique())

52

In [52]:
resultDF.drop_duplicates(keep="first", inplace=True)

In [53]:
resultDF

,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
36,6,2,1834,743,188,32,96.690132,12.01.2023,-1723,2022,...,0.0,0.0,0.0,0.000000,454.001101,0.000000,0.000000,0.0,68.jpeg,0
175,6,0,1834,743,188,32,96.690132,12.01.2023,-1723,2022,...,0.0,0.0,0.0,0.000000,454.001101,0.000000,0.000000,0.0,68.jpeg,0
321,27,1,2229,2351,155,25,95.671471,28-03-2023:,-2307,2384,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,288.006944,0.0,55.jpeg,0
409,8,0,1806,287,249,28,73.119611,"Mar 28, 2023 16:37 to",-1586,2055,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,55.jpeg,0
433,8,0,1806,473,153,32,80.006772,"‘Mar 28, 2023",-1900,1959,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,55.jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,13,0,833,1553,359,31,66.729904,:'2022-10-19 16:51:00,-1172,1192,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,51.jpeg,0
36,4,5,975,665,139,31,96.866020,14-Jan-2023,-912,1114,...,0.0,0.0,0.0,814.000000,868.027649,0.000000,0.000000,0.0,69.jpeg,0
144,4,0,923,661,182,39,96.178218,"net, 14-Jan-2023",-912,1105,...,0.0,0.0,0.0,805.009938,860.958187,0.000000,0.000000,0.0,69.jpeg,0
16,2,8,577,482,175,31,84.334061,04/11/2022:,-736,752,...,0.0,0.0,0.0,0.000000,431.236594,685.011679,0.000000,0.0,62.jpeg,0


In [54]:
resultDF.reset_index(inplace=True)

In [55]:
resultDF.drop(columns=["index", "nextDist"], inplace=True)

In [56]:
resultDF.head()

,block_num,line_num,left,top,width,height,conf,text,x,y,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
0,6,2,1834,743,188,32,96.690132,12.01.2023,2022,775,...,0.0,0.0,0.0,0.0,454.001101,0.0,0.000000,0.0,68.jpeg,0
1,6,0,1834,743,188,32,96.690132,12.01.2023,2022,775,...,0.0,0.0,0.0,0.0,454.001101,0.0,0.000000,0.0,68.jpeg,0
2,27,1,2229,2351,155,25,95.671471,28-03-2023:,2384,2376,...,0.0,0.0,0.0,0.0,0.000000,0.0,288.006944,0.0,55.jpeg,0
3,8,0,1806,287,249,28,73.119611,"Mar 28, 2023 16:37 to",2055,315,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,55.jpeg,0
4,8,0,1806,473,153,32,80.006772,"‘Mar 28, 2023",1959,505,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,55.jpeg,0


In [57]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset/')

In [58]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset'

In [59]:
resultDF.to_csv("tesseract.csv")